In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading data function for function2
def load_data():
    A = np.loadtxt("data/fun2_A.txt").reshape((500, 100))
    b = np.loadtxt("data/fun2_b.txt").reshape(-1, 1)
    c = np.loadtxt("data/fun2_c.txt").reshape(-1, 1)
    return A, b, c


# Initialize coefficient and start point
A, b, c = load_data()
x0_f1 = np.random.rand(100)
x0_f2 = np.abs(np.random.rand(100))
x0_f3 = np.array([0.0, 0.0])

In [ ]:
# Define function1
def function1(x):
    return np.sum(np.arange(1, len(x) + 1) * x**2)


def grad_function1(x):
    return 2 * np.arange(1, len(x) + 1) * x

In [ ]:
# Define function2
def function2(x, A, b, c):
    Ax = A @ x
    return c.T @ x - np.sum(np.log(np.clip(b.flatten() - Ax, 1e-8, None)))


def grad_function2(x, A, b, c):
    Ax = A @ x
    term = 1 / (b.flatten() - Ax)
    return c.flatten() + A.T @ term


def hessian_function2(x, A, b):
    Ax = A @ x
    D = np.diagflat(1 / np.clip((b.flatten() - Ax) ** 2, 1e-10, None))
    H = A.T @ D @ A
    H += 1e-6 * np.eye(H.shape[0])
    return H

In [ ]:
# Define function3
def function3(x):
    return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2


def grad_function3(x):
    return np.array(
        [-400 * x[0] * (x[1] - x[0] ** 2) - 2 * (1 - x[0]), 200 * (x[1] - x[0] ** 2)]
    )


def hessian_function3(x):
    return np.array(
        [[1200 * x[0] ** 2 - 400 * x[1] + 2, -400 * x[0]], [-400 * x[0], 200]]
    )

In [ ]:
# Optimization method 1: GD
def gradient_descent(func, grad_func, x0, alpha=0.001, tol=1e-6, max_iter=200):
    x = x0
    history = []
    for i in range(max_iter):
        grad = grad_func(x)
        history.append(func(x))
        x = x - alpha * grad
        if np.linalg.norm(grad) < tol:
            break
    return x, history

In [ ]:
# ----- Function1 -----
gd_result_f1, gd_history_f1 = gradient_descent(function1, grad_function1, x0_f1)
print("Function1 Results:")
print("Gradient Descent:", gd_result_f1)

# Plot for function1
plt.figure(figsize=(10, 6))
plt.plot(gd_history_f1, label="Gradient Descent")
plt.xlabel("Interation")
plt.ylabel("Function Value")
plt.title("Covergence of Function1")
plt.legend()
plt.show()